## Why Instance Registries?

Some components need configuration that can't easily be passed at instantiation time. For example, scorers often need:
- A configured `chat_target` for LLM-based scoring
- Specific prompt templates
- Other dependencies

Instance registries let initializers register fully-configured instances that are ready to use.

## Listing Available Instances

Use `get_names()` to see registered instances, or `list_metadata()` for details.

In [ ]:
from pyrit.prompt_target import OpenAIChatTarget
from pyrit.registry import ScorerRegistry
from pyrit.score import SelfAskRefusalScorer
from pyrit.setup import IN_MEMORY, initialize_pyrit_async

await initialize_pyrit_async(memory_db_type=IN_MEMORY)  # type: ignore

# Get the registry singleton
registry = ScorerRegistry.get_instance()

# Register a scorer instance for demonstration
chat_target = OpenAIChatTarget()
refusal_scorer = SelfAskRefusalScorer(chat_target=chat_target)
registry.register_instance(refusal_scorer)

# List what's available
names = registry.get_names()
print(f"Registered scorers: {names}")

## Getting an Instance

Use `get()` to retrieve a pre-configured instance by name. The instance is ready to use immediately.

In [ ]:
# Get the first registered scorer
if names:
    scorer_name = names[0]
    scorer = registry.get(scorer_name)
    print(f"Retrieved scorer: {scorer}")
    print(f"Scorer type: {type(scorer).__name__}")

## Inspecting Metadata

Scorer metadata includes the scorer type and identifier for tracking.

In [ ]:
# Get metadata for all registered scorers
metadata = registry.list_metadata()
for item in metadata:
    print(f"\n{item['name']}:")
    print(f"  Class: {item['class_name']}")
    print(f"  Type: {item['scorer_type']}")
    print(f"  Description: {item['description'][:60]}...")

## Filtering by Type

The `ScorerRegistry` provides a helper to filter scorers by type.

In [ ]:
# Get only true/false scorers
true_false_scorers = registry.list_metadata_by_type("true_false")
print(f"True/False scorers: {[m['name'] for m in true_false_scorers]}")

# Get only float scale scorers
float_scorers = registry.list_metadata_by_type("float_scale")
print(f"Float scale scorers: {[m['name'] for m in float_scorers]}")

## Key Difference from Class Registries

| Aspect | Class Registry | Instance Registry |
|--------|----------------|-------------------|
| Stores | Classes (Type[T]) | Instances (T) |
| Registration | Automatic discovery | Explicit via `register_instance()` |
| Returns | Class to instantiate | Ready-to-use instance |
| Use case | User provides config | Pre-configured by initializer |